In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

# 1. EDA

In [14]:
dataset = pd.read_csv(r'C:\Users\vickj\OneDrive\Documents\Foundations of data engineering (Mtech)\03_Udemy_DeepLearning\ANN\Dataset\Part 1 - Artificial Neural Networks\Churn_Modelling.csv')

In [15]:
dataset.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [16]:
Gender_table1 = dataset.pivot_table(index='Gender', values='CustomerId', aggfunc='count')
print(Gender_table1)

        CustomerId
Gender            
Female        4543
Male          5457


In [17]:
Geography_table1 = dataset.pivot_table(index='Geography', values='CustomerId', aggfunc='count')
print(Geography_table1)

           CustomerId
Geography            
France           5014
Germany          2509
Spain            2477


# 2. Data Preprocessing

In [18]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

# .values will convert dataset values to numpy Array
# iloc means integer location based indexing 

In [19]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [20]:
#making the gender coloumn binary its also called "Label Encoding"

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [21]:
#One Hot Encoding the "Geography" column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Training set and Test set

In [22]:
# Splitting the dataset into the Training set and Test set
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Feature Scaling

In [23]:
# Feature Scaling
def scale_features(X_train, X_test):
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    X_train_scaled = (X_train - mean) / std
    X_test_scaled = (X_test - mean) / std
    return X_train_scaled, X_test_scaled

# Creating ANN 

In [28]:
import numpy as np

X_train = X_train.astype(float)
y_train = y_train.astype(float)


class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x.astype(float)))  # Explicitly cast to float

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        # Forward propagation
        self.hidden_output = self.sigmoid(np.dot(X, self.weights_input_hidden) + self.bias_hidden)
        self.final_output = self.sigmoid(np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output)
        return self.final_output

    def backward(self, X, y, learning_rate):
        # Backward propagation
        error = y - self.final_output
        output_delta = error * self.sigmoid_derivative(self.final_output)

        hidden_layer_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_layer_delta = hidden_layer_error * self.sigmoid_derivative(self.hidden_output)

        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        self.weights_input_hidden += X.T.dot(hidden_layer_delta) * learning_rate
        self.bias_hidden += np.sum(hidden_layer_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            # Forward and backward for each epoch
            output = self.forward(X)
            self.backward(X, y, learning_rate)

            # Print loss for every 1000 epochs
            if epoch % 1000 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        # Make predictions
        return self.forward(X)

# Example usage:
# Assuming you have your dataset X and corresponding labels y
# (X_train, y_train, X_test, y_test for training and testing)
input_size = X_train.shape[1]
hidden_size = 10
output_size = 1

# Create a neural network
nn = NeuralNetwork(input_size, hidden_size, output_size)

# Train the neural network
nn.train(X_train, y_train.reshape(-1, 1), epochs=10000, learning_rate=0.01)

# Make predictions on the test set
predictions = nn.predict(X_test)

# Threshold predictions (assuming binary classification)
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

# Evaluate accuracy or other metrics
accuracy = np.mean(binary_predictions == y_test.reshape(-1, 1))
print(f"Accuracy: {accuracy}")


C:\Users\vickj\AppData\Local\Temp\ipykernel_21648\2912591432.py:16: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x.astype(float)))  # Explicitly cast to float


Epoch 0, Loss: 0.7795829282479796
Epoch 1000, Loss: 0.2058749994558309
Epoch 2000, Loss: 0.20587499945575985
Epoch 3000, Loss: 0.20587499945568874
Epoch 4000, Loss: 0.20587499945561763
Epoch 5000, Loss: 0.20587499945554652
Epoch 6000, Loss: 0.20587499945547538
Epoch 7000, Loss: 0.2058749994554042
Epoch 8000, Loss: 0.20587499945533302
Epoch 9000, Loss: 0.20587499945526178
Accuracy: 0.805


## Training the ANN on the Training set

In [29]:
print(nn.predict(np.array([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


C:\Users\vickj\AppData\Local\Temp\ipykernel_21648\2912591432.py:16: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x.astype(float)))  # Explicitly cast to float


# Predicting the Test set results

In [30]:
# Assuming you have your dataset X_test and corresponding labels y_test
# (X_test, y_test for testing)

# Make predictions on the test set
predictions = nn.predict(X_test)

# Threshold predictions (assuming binary classification)
threshold = 0.5
y_pred = (predictions > threshold).astype(int)

# Print the predictions and actual labels side by side
result = np.concatenate((binary_predictions.reshape(len(binary_predictions), 1), y_test.reshape(len(y_test), 1)), 1)
print("Predictions vs Actual Labels:")
print(result)


Predictions vs Actual Labels:
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 1]
 [0 1]
 [0 0]]


C:\Users\vickj\AppData\Local\Temp\ipykernel_21648\2912591432.py:16: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x.astype(float)))  # Explicitly cast to float


# Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1610    0]
 [ 390    0]]


0.805